# JEE Cutoff Prediction Model
## Phase 3: Model Building and Training

**Objective**: Build and train XGBoost model to predict JEE cutoffs

**Key Tasks**:
- Load feature-engineered data
- Split data using time-series approach
- Train XGBoost model
- Hyperparameter tuning
- Evaluate model performance
- Analyze feature importance

**Course Alignment**: 
- Unit 4: Regression
- Unit 5: Ensemble Learning (XGBoost)

## Step 1: Import Libraries

In [6]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import pickle
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")
print(f"XGBoost version: {xgb.__version__}")

✅ Libraries imported successfully!
XGBoost version: 3.1.1


## Step 2: Load Feature-Engineered Data

In [7]:
# Load the model-ready dataset
df = pd.read_csv('cutoffs_model_ready.csv')

print("✅ Data loaded successfully!")
print(f"📊 Shape: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"\nFirst few rows:")
print(df.head())

# Load feature names
feature_info = pd.read_csv('feature_names.csv')
feature_columns = feature_info['feature_name'].tolist()

print(f"\n📋 Total features for modeling: {len(feature_columns)}")
print(f"\n✓ Feature list loaded from Phase 2")

✅ Data loaded successfully!
📊 Shape: 79521 rows × 25 columns

First few rows:
   institute_encoded  branch_encoded  quota_encoded  seat_type_encoded  \
0                  0               1              0                  0   
1                  0               1              0                  0   
2                  0               1              0                  0   
3                  0               1              0                  0   
4                  0               1              0                  0   

   gender_encoded  branch_demand_category_encoded  cutoff_prev_1yr  \
0               1                               1          54330.8   
1               1                               1           7839.0   
2               1                               1          11172.0   
3               1                               1          13870.0   
4               1                               1          13007.0   

   cutoff_prev_2yr  cutoff_prev_3yr  cutoff_mean_3yr  cu

## Step 3: Prepare Features and Target

Separate features (X) and target variable (y)

In [8]:
# Define features and target
X = df[feature_columns].copy()
y = df['cutoff'].copy()

# Keep metadata for later analysis
metadata = df[['seat_id', 'institute', 'branch']].copy()

print("✅ Features and target prepared!")
print(f"\n📊 Feature matrix (X) shape: {X.shape}")
print(f"📊 Target vector (y) shape: {y.shape}")
print(f"\n📈 Target variable statistics:")
print(y.describe())

✅ Features and target prepared!

📊 Feature matrix (X) shape: (79521, 21)
📊 Target vector (y) shape: (79521,)

📈 Target variable statistics:
count    7.952100e+04
mean     1.930295e+04
std      4.066367e+05
min      1.000000e+00
25%      1.397000e+03
50%      4.587000e+03
75%      1.300800e+04
max      1.008650e+08
Name: cutoff, dtype: float64


## Step 4: Extract Year Information

Get the year column for time-series splitting (must be done before train-test split)

In [9]:
# Get year column for splitting (save before any data manipulation)
years = df['year'].copy()

print("✅ Year information extracted!")
print(f"\n📊 Year distribution:")
print(years.value_counts().sort_index())

✅ Year information extracted!

📊 Year distribution:
year
2018     7157
2019     8745
2020     9340
2021     9452
2022    10023
2023    10842
2024    11688
2025    12274
Name: count, dtype: int64


## Step 5: Check for Missing Values and Handle Them

Before training, we need to check and handle any missing values in the data

In [10]:
# Check for missing values in the entire dataset
print("🔍 Checking for missing values...")
print("\n" + "="*80)
print("📊 MISSING VALUES IN DATASET")
print("="*80)

missing_before = X.isnull().sum()
missing_features = missing_before[missing_before > 0]

if len(missing_features) > 0:
    print(f"\n⚠️  Found {len(missing_features)} features with missing values:")
    for feature, count in missing_features.items():
        pct = (count / len(X)) * 100
        print(f"  {feature}: {count:,} missing ({pct:.2f}%)")
    
    print(f"\n🔄 Handling missing values...")
    
    # Fill missing values with median for numeric features
    # This is safe for our use case (lag features, statistical features)
    for col in missing_features.index:
        median_value = X[col].median()
        X[col].fillna(median_value, inplace=True)
        print(f"  ✓ Filled {col} with median: {median_value:.2f}")
    
    # Verify no more missing values
    missing_after = X.isnull().sum().sum()
    print(f"\n✅ Missing values handled!")
    print(f"  Before: {missing_before.sum():,} missing values")
    print(f"  After: {missing_after} missing values")
else:
    print("\n✅ No missing values found!")

print("="*80)

🔍 Checking for missing values...

📊 MISSING VALUES IN DATASET

⚠️  Found 3 features with missing values:
  cutoff_prev_1yr: 125 missing (0.16%)
  cutoff_prev_2yr: 417 missing (0.52%)
  cutoff_prev_3yr: 978 missing (1.23%)

🔄 Handling missing values...
  ✓ Filled cutoff_prev_1yr with median: 5336.47
  ✓ Filled cutoff_prev_2yr with median: 5812.23
  ✓ Filled cutoff_prev_3yr with median: 5643.00

✅ Missing values handled!
  Before: 1,520 missing values
  After: 0 missing values


## Step 6: Time-Series Train-Test Split

**Important**: We use time-based split, NOT random split!
- Train: 2018-2023 (6 years)
- Test: 2024 (1 year)

This prevents data leakage (using future data to predict past)

In [11]:
# Time-based split: Train on 2018-2023, Test on 2024
train_mask = years < 2024
test_mask = years == 2024

X_train = X[train_mask].copy()
y_train = y[train_mask].copy()
X_test = X[test_mask].copy()
y_test = y[test_mask].copy()

# Metadata for analysis
train_metadata = metadata[train_mask].copy()
test_metadata = metadata[test_mask].copy()

print("✅ Time-series split completed!")
print(f"\n📊 Training set:")
print(f"  - Years: {years[train_mask].min()} to {years[train_mask].max()}")
print(f"  - Samples: {len(X_train):,}")
print(f"  - Shape: {X_train.shape}")

print(f"\n📊 Test set:")
print(f"  - Year: {years[test_mask].unique()[0]}")
print(f"  - Samples: {len(X_test):,}")
print(f"  - Shape: {X_test.shape}")

print(f"\n📈 Train-Test split ratio: {len(X_train)/len(X)*100:.1f}% - {len(X_test)/len(X)*100:.1f}%")

# Final check for NaN values in train and test sets
print(f"\n🔍 Final data quality check:")
print(f"  - NaN in X_train: {X_train.isnull().sum().sum()}")
print(f"  - NaN in X_test: {X_test.isnull().sum().sum()}")
print(f"  - NaN in y_train: {y_train.isnull().sum()}")
print(f"  - NaN in y_test: {y_test.isnull().sum()}")

✅ Time-series split completed!

📊 Training set:
  - Years: 2018 to 2023
  - Samples: 55,559
  - Shape: (55559, 21)

📊 Test set:
  - Year: 2024
  - Samples: 11,688
  - Shape: (11688, 21)

📈 Train-Test split ratio: 69.9% - 14.7%

🔍 Final data quality check:
  - NaN in X_train: 0
  - NaN in X_test: 0
  - NaN in y_train: 0
  - NaN in y_test: 0


## Step 7: Baseline Model - Simple Linear Regression

Create a baseline to compare XGBoost performance against

In [12]:
from sklearn.linear_model import LinearRegression

print("🔄 Training baseline Linear Regression model...")

# Train simple linear regression
baseline_model = LinearRegression()
baseline_model.fit(X_train, y_train)

# Predictions
y_train_pred_baseline = baseline_model.predict(X_train)
y_test_pred_baseline = baseline_model.predict(X_test)

# Evaluate baseline
train_mae_baseline = mean_absolute_error(y_train, y_train_pred_baseline)
test_mae_baseline = mean_absolute_error(y_test, y_test_pred_baseline)
train_rmse_baseline = np.sqrt(mean_squared_error(y_train, y_train_pred_baseline))
test_rmse_baseline = np.sqrt(mean_squared_error(y_test, y_test_pred_baseline))
train_r2_baseline = r2_score(y_train, y_train_pred_baseline)
test_r2_baseline = r2_score(y_test, y_test_pred_baseline)

print("\n✅ Baseline model trained!")
print("\n" + "="*80)
print("📊 BASELINE MODEL PERFORMANCE (Linear Regression)")
print("="*80)
print(f"\nTraining Set:")
print(f"  MAE:  {train_mae_baseline:,.2f} ranks")
print(f"  RMSE: {train_rmse_baseline:,.2f} ranks")
print(f"  R²:   {train_r2_baseline:.4f}")

print(f"\nTest Set (2024):")
print(f"  MAE:  {test_mae_baseline:,.2f} ranks")
print(f"  RMSE: {test_rmse_baseline:,.2f} ranks")
print(f"  R²:   {test_r2_baseline:.4f}")
print("="*80)

🔄 Training baseline Linear Regression model...

✅ Baseline model trained!

📊 BASELINE MODEL PERFORMANCE (Linear Regression)

Training Set:
  MAE:  24,254.53 ranks
  RMSE: 438,059.58 ranks
  R²:   0.0541

Test Set (2024):
  MAE:  27,169.44 ranks
  RMSE: 263,666.55 ranks
  R²:   0.1922


## Step 8: Build Initial XGBoost Model

Train XGBoost with default parameters first

In [ ]:
print("🔄 Training initial XGBoost model...\n")

# Create XGBoost regressor with initial parameters
xgb_initial = xgb.XGBRegressor(
    objective='reg:squarederror',  # Regression task
    n_estimators=100,               # Number of trees
    max_depth=6,                    # Tree depth
    learning_rate=0.1,              # Learning rate
    random_state=42,
    n_jobs=-1,                      # Use all CPU cores
    eval_metric='mae'               # Evaluation metric
)

# Train the model
xgb_initial.fit(
    X_train, 
    y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=False
)

# Predictions
y_train_pred_initial = xgb_initial.predict(X_train)
y_test_pred_initial = xgb_initial.predict(X_test)

# Evaluate
train_mae_initial = mean_absolute_error(y_train, y_train_pred_initial)
test_mae_initial = mean_absolute_error(y_test, y_test_pred_initial)
train_rmse_initial = np.sqrt(mean_squared_error(y_train, y_train_pred_initial))
test_rmse_initial = np.sqrt(mean_squared_error(y_test, y_test_pred_initial))
train_r2_initial = r2_score(y_train, y_train_pred_initial)
test_r2_initial = r2_score(y_test, y_test_pred_initial)

print("✅ Initial XGBoost model trained!")
print("\n" + "="*80)
print("📊 INITIAL XGBOOST PERFORMANCE (Default Parameters)")
print("="*80)
print(f"\nTraining Set:")
print(f"  MAE:  {train_mae_initial:,.2f} ranks")
print(f"  RMSE: {train_rmse_initial:,.2f} ranks")
print(f"  R²:   {train_r2_initial:.4f}")

print(f"\nTest Set (2024):")
print(f"  MAE:  {test_mae_initial:,.2f} ranks")
print(f"  RMSE: {test_rmse_initial:,.2f} ranks")
print(f"  R²:   {test_r2_initial:.4f}")

print(f"\n💡 Improvement over baseline:")
print(f"  MAE improvement: {test_mae_baseline - test_mae_initial:,.2f} ranks ({(test_mae_baseline - test_mae_initial)/test_mae_baseline*100:.1f}%)")
print(f"  RMSE improvement: {test_rmse_baseline - test_rmse_initial:,.2f} ranks ({(test_rmse_baseline - test_rmse_initial)/test_rmse_baseline*100:.1f}%)")
print("="*80)

🔄 Training initial XGBoost model...



TypeError: XGBModel.fit() got an unexpected keyword argument 'eval_metric'

## Step 9: Hyperparameter Tuning

Use RandomizedSearchCV to find optimal hyperparameters

In [ ]:
print("🔄 Starting hyperparameter tuning...\n")
print("⏳ This may take several minutes...\n")

# Define parameter grid
param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 4, 5, 6, 7],
    'learning_rate': [0.01, 0.05, 0.1, 0.15],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.2]
}

# Create base model
xgb_base = xgb.XGBRegressor(
    objective='reg:squarederror',
    random_state=42,
    n_jobs=-1
)

# Time series cross-validation
# We'll use 3 splits on training data
tscv = TimeSeriesSplit(n_splits=3)

# Randomized search (faster than GridSearch)
random_search = RandomizedSearchCV(
    estimator=xgb_base,
    param_distributions=param_grid,
    n_iter=30,  # Try 30 random combinations
    scoring='neg_mean_absolute_error',
    cv=tscv,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

# Fit the random search
random_search.fit(X_train, y_train)

print("\n✅ Hyperparameter tuning completed!")
print(f"\n🏆 Best parameters found:")
for param, value in random_search.best_params_.items():
    print(f"  {param}: {value}")

print(f"\n📊 Best cross-validation MAE: {-random_search.best_score_:,.2f} ranks")

## Step 10: Train Final Optimized Model

Train XGBoost with the best hyperparameters found

In [ ]:
print("🔄 Training final optimized XGBoost model...\n")

# Get the best model from random search
xgb_final = random_search.best_estimator_

# Set eval_metric for the final model
xgb_final.set_params(eval_metric='mae')

# Train on full training data with evaluation sets
xgb_final.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=False
)

# Predictions
y_train_pred_final = xgb_final.predict(X_train)
y_test_pred_final = xgb_final.predict(X_test)

# Evaluate final model
train_mae_final = mean_absolute_error(y_train, y_train_pred_final)
test_mae_final = mean_absolute_error(y_test, y_test_pred_final)
train_rmse_final = np.sqrt(mean_squared_error(y_train, y_train_pred_final))
test_rmse_final = np.sqrt(mean_squared_error(y_test, y_test_pred_final))
train_r2_final = r2_score(y_train, y_train_pred_final)
test_r2_final = r2_score(y_test, y_test_pred_final)

# Calculate MAPE (Mean Absolute Percentage Error)
train_mape_final = np.mean(np.abs((y_train - y_train_pred_final) / y_train)) * 100
test_mape_final = np.mean(np.abs((y_test - y_test_pred_final) / y_test)) * 100

print("✅ Final optimized model trained!")
print("\n" + "="*80)
print("📊 FINAL OPTIMIZED XGBOOST PERFORMANCE")
print("="*80)
print(f"\nTraining Set:")
print(f"  MAE:   {train_mae_final:,.2f} ranks")
print(f"  RMSE:  {train_rmse_final:,.2f} ranks")
print(f"  R²:    {train_r2_final:.4f}")
print(f"  MAPE:  {train_mape_final:.2f}%")

print(f"\nTest Set (2024):")
print(f"  MAE:   {test_mae_final:,.2f} ranks")
print(f"  RMSE:  {test_rmse_final:,.2f} ranks")
print(f"  R²:    {test_r2_final:.4f}")
print(f"  MAPE:  {test_mape_final:.2f}%")

print(f"\n💡 Improvement over baseline (Linear Regression):")
print(f"  MAE improvement:  {test_mae_baseline - test_mae_final:,.2f} ranks ({(test_mae_baseline - test_mae_final)/test_mae_baseline*100:.1f}%)")
print(f"  RMSE improvement: {test_rmse_baseline - test_rmse_final:,.2f} ranks ({(test_rmse_baseline - test_rmse_final)/test_rmse_baseline*100:.1f}%)")

print(f"\n💡 Improvement over initial XGBoost:")
print(f"  MAE improvement:  {test_mae_initial - test_mae_final:,.2f} ranks ({(test_mae_initial - test_mae_final)/test_mae_initial*100:.1f}%)")
print(f"  RMSE improvement: {test_rmse_initial - test_rmse_final:,.2f} ranks ({(test_rmse_initial - test_rmse_final)/test_rmse_initial*100:.1f}%)")
print("="*80)

## Step 11: Feature Importance Analysis

Understand which features contribute most to predictions

In [ ]:
# Get feature importance
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': xgb_final.feature_importances_
}).sort_values('importance', ascending=False)

print("📊 TOP 15 MOST IMPORTANT FEATURES")
print("="*80)
print(feature_importance.head(15).to_string(index=False))
print("="*80)

# Visualize feature importance
plt.figure(figsize=(12, 8))
top_features = feature_importance.head(15)
plt.barh(range(len(top_features)), top_features['importance'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Importance Score', fontsize=12)
plt.title('Top 15 Feature Importance in XGBoost Model', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\n✅ Feature importance analysis completed!")

## Step 12: Prediction Analysis - Actual vs Predicted

Visualize how well the model predicts cutoffs

In [ ]:
# Create visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Actual vs Predicted (Test Set)
axes[0].scatter(y_test, y_test_pred_final, alpha=0.5, s=10)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual Cutoff Rank', fontsize=12)
axes[0].set_ylabel('Predicted Cutoff Rank', fontsize=12)
axes[0].set_title('Actual vs Predicted Cutoffs (Test Set 2024)', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Plot 2: Residual plot
residuals = y_test - y_test_pred_final
axes[1].scatter(y_test_pred_final, residuals, alpha=0.5, s=10)
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Predicted Cutoff Rank', fontsize=12)
axes[1].set_ylabel('Residuals (Actual - Predicted)', fontsize=12)
axes[1].set_title('Residual Plot', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Prediction visualization completed!")

## Step 13: Error Analysis by Category

Analyze model performance across different branches and institute tiers

In [ ]:
# Create results dataframe for test set
test_results = test_metadata.copy()
test_results['actual'] = y_test.values
test_results['predicted'] = y_test_pred_final
test_results['error'] = np.abs(y_test.values - y_test_pred_final)
test_results['pct_error'] = (test_results['error'] / test_results['actual'] * 100)

print("📊 ERROR ANALYSIS BY BRANCH")
print("="*80)
branch_errors = test_results.groupby('branch').agg({
    'error': ['mean', 'median', 'std'],
    'pct_error': 'mean',
    'actual': 'count'
}).round(2)
branch_errors.columns = ['MAE', 'Median Error', 'Std Error', 'MAPE %', 'Count']
branch_errors = branch_errors.sort_values('MAE', ascending=False).head(10)
print(branch_errors)

print("\n" + "="*80)
print("📊 BEST PERFORMING BRANCHES (Lowest Error)")
print("="*80)
best_branches = test_results.groupby('branch').agg({
    'error': 'mean',
    'actual': 'count'
}).round(2)
best_branches.columns = ['MAE', 'Count']
best_branches = best_branches[best_branches['Count'] >= 10].sort_values('MAE').head(10)
print(best_branches)

print("\n✅ Error analysis completed!")

## Step 14: Sample Predictions

Show some example predictions for different institutes and branches

In [ ]:
# Show sample predictions
sample_results = test_results[['institute', 'branch', 'actual', 'predicted', 'error', 'pct_error']].copy()

print("📋 SAMPLE PREDICTIONS (Best Predictions)")
print("="*80)
print(sample_results.nsmallest(10, 'error').to_string(index=False))

print("\n" + "="*80)
print("📋 SAMPLE PREDICTIONS (Worst Predictions)")
print("="*80)
print(sample_results.nlargest(10, 'error').to_string(index=False))

print("\n" + "="*80)
print("📋 RANDOM SAMPLE PREDICTIONS")
print("="*80)
print(sample_results.sample(10).to_string(index=False))

## Step 15: Save the Trained Model

Save the model for future predictions and deployment

In [ ]:
# Save the final model
with open('xgboost_cutoff_model.pkl', 'wb') as f:
    pickle.dump(xgb_final, f)

print("✅ Model saved successfully!")
print(f"📁 File: xgboost_cutoff_model.pkl")

# Save model performance metrics
performance_metrics = {
    'model': 'XGBoost Regressor',
    'training_years': '2018-2023',
    'test_year': '2024',
    'test_mae': test_mae_final,
    'test_rmse': test_rmse_final,
    'test_r2': test_r2_final,
    'test_mape': test_mape_final,
    'n_features': len(feature_columns),
    'best_params': random_search.best_params_
}

import json
with open('model_performance.json', 'w') as f:
    json.dump(performance_metrics, f, indent=4)

print(f"\n✅ Performance metrics saved!")
print(f"📁 File: model_performance.json")

# Save feature importance
feature_importance.to_csv('feature_importance.csv', index=False)
print(f"\n✅ Feature importance saved!")
print(f"📁 File: feature_importance.csv")

## Step 16: Model Summary Report

In [ ]:
print("\n" + "="*80)
print("🎉 PHASE 3: MODEL BUILDING COMPLETED!")
print("="*80)

print("\n📊 FINAL MODEL SUMMARY")
print("="*80)
print(f"Model Type: XGBoost Regressor (Ensemble Learning)")
print(f"Training Data: 2018-2023 ({len(X_train):,} samples)")
print(f"Test Data: 2024 ({len(X_test):,} samples)")
print(f"Number of Features: {len(feature_columns)}")

print(f"\n🏆 BEST HYPERPARAMETERS:")
for param, value in random_search.best_params_.items():
    print(f"  {param}: {value}")

print(f"\n📈 PERFORMANCE METRICS (Test Set 2024):")
print(f"  Mean Absolute Error (MAE):  {test_mae_final:,.2f} ranks")
print(f"  Root Mean Squared Error:    {test_rmse_final:,.2f} ranks")
print(f"  R² Score:                   {test_r2_final:.4f}")
print(f"  Mean Absolute % Error:      {test_mape_final:.2f}%")

print(f"\n💡 INTERPRETATION:")
print(f"  - On average, predictions are off by {test_mae_final:,.0f} ranks")
print(f"  - Model explains {test_r2_final*100:.1f}% of variance in cutoffs")
print(f"  - Percentage error is {test_mape_final:.1f}% on average")

print(f"\n🔝 TOP 5 MOST IMPORTANT FEATURES:")
for i, row in feature_importance.head(5).iterrows():
    print(f"  {row['feature']}: {row['importance']:.4f}")

print(f"\n📁 FILES CREATED:")
print(f"  1. xgboost_cutoff_model.pkl - Trained XGBoost model")
print(f"  2. model_performance.json - Performance metrics")
print(f"  3. feature_importance.csv - Feature importance scores")

print("\n" + "="*80)
print("✅ Ready for Phase 4: Making Predictions!")
print("="*80)